# Model:Swin Dataset:224 with border


In [ ]:
!pip install timm

In [ ]:
import math
import random
import time
import warnings
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers as T
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import pandas as pd
import torchvision.models as models
import timm

import torch.nn.functional as F
import torchvision.models as models

def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed = 2021
seed_torch(seed)

In [ ]:
train_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')

## Modelの構築
Swinネットを利用

In [ ]:
class Create_Model(nn.Module):
    def __init__(self):
        super(Create_Model,self).__init__()
        self.swin = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True,num_classes=0, in_chans=3)
        num_features = self.swin.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, 100)
            ,nn.Dropout(0.5), nn.Linear(100, 1)
        )
    def forward(self,x):
        x = self.swin(x)
        x = self.fc(x)
        return x

In [ ]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image 
import torchvision.transforms as T
from tqdm import tqdm
from torchvision.io import read_image

class CustomDataSet(Dataset):
    def __init__(self,df):
        self.img_list =  df['Id'].to_list()
        self.table_list = df[df.columns[1:13]]
        self.label_list = df[df.columns[-1]].to_list()
        self.preprocess = nn.Sequential(
            T.ConvertImageDtype(torch.float32),
            T.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
        ).to(torch.device('cuda'))
   
    def __getitem__(self,idx):
        path = "../input/petfinder224/train_224/"+self.img_list[idx]+'.jpg'
        img = read_image(path)
        img = self.preprocess(img)
        tbl = torch.tensor(list(self.table_list.iloc[idx])).to(torch.float32)
        label = torch.tensor(self.label_list[idx]/100).to(torch.float32)
        return img,tbl,label
    def __len__(self):
        return len(self.img_list)

    

In [ ]:
FOLD_NUM = 5

## 実験内容
- 224×224
- ボーダーラインつき
- Swinモデル

In [ ]:

from sklearn.model_selection import KFold
import torch.optim as opt
import numpy as np
import time
import gc

kf = KFold(n_splits=FOLD_NUM, random_state=None, shuffle=False)

BATCH_SIZE = 96
EPOCH = 50
DEVICE = 'cuda'
STOP_NUM = 3
criterion_mse = nn.MSELoss()
criterion_bce = nn.BCEWithLogitsLoss()

best_score_list = []
for k,(train_index, test_index) in enumerate(kf.split(train_df['Id'])):
    print(f"========={k+1}-FOLD=========")
    model = Create_Model().to(DEVICE)
    
    train_dataset = CustomDataSet(train_df.loc[train_index])
    val_dataset = CustomDataSet(train_df.loc[test_index])
    train_loader = DataLoader(train_dataset,batch_size = BATCH_SIZE,shuffle = True,num_workers = 8)
    val_loader = DataLoader(val_dataset,batch_size = BATCH_SIZE,shuffle = False,num_workers = 8)
   

    
    optimizer = opt.AdamW(params = model.parameters(),lr = 1e-5)
    scheduler = opt.lr_scheduler.CosineAnnealingWarmRestarts(optimizer , 100 , eta_min=1e-4)
    best_loss = 99999
    stop_counter = 0
    
    for e in range(EPOCH):
        model.train()
        train_total_loss = 0
        train_score = 0
        counter = 0
        
        for n,(img,tbl,label) in enumerate(train_loader):
            img = img.to(DEVICE)
            tbl = tbl.to(DEVICE)
            label = label.to(DEVICE)
            optimizer.zero_grad()
            
            output = model(img)[:,0]
            
            loss = criterion_bce(output , label)
            score = criterion_mse(F.sigmoid(output)*100 , label*100).detach()
            
            
            train_total_loss = (train_total_loss * n + loss.detach().item())/(n+1)
            train_score =  (train_score * counter + score.item()*output.shape[0])/(counter + output.shape[0])
            
            counter += output.shape[0]
            loss.backward()
            optimizer.step()
            print('\rTRAIN EPOCH[{:03}/{:03}] ITR[{:04}/{:04}] LOSS:{:.5} SCORE:{:.5}'.format(e+1,EPOCH,n+1,len(train_loader),train_total_loss,np.sqrt(train_score)),end = "")
        scheduler.step()
        print()
        val_total_loss = 0
        val_score = 0
        counter = 0
        model.eval()
        with torch.no_grad():
            for n,(img,tbl,label) in enumerate(val_loader):
                img = img.to(DEVICE)
                tbl = tbl.to(DEVICE)
                label = label.to(DEVICE)
            
                output = model(img)[:,0]

                loss = criterion_bce(output , label)
                score = criterion_mse(F.sigmoid(output)*100 , label*100).detach()
                
                
                val_total_loss = (val_total_loss * n + loss.item())/(n+1)
                
                val_score =  (val_score * counter + score.item()*output.shape[0])/(counter + output.shape[0])
                
                counter += output.shape[0]
                print('\rVAL   EPOCH[{:03}/{:03}] ITR[{:04}/{:04}] LOSS:{:.5} SCORE:{:.5}'.format(e+1,EPOCH,n+1,len(val_loader),val_total_loss,np.sqrt(val_score)),end = "")
        print()
        if(best_loss > val_total_loss):
            best_loss = val_total_loss
            model_path = f'{k+1}-fold.pth'
            torch.save(model.state_dict(), model_path)
            stop_counter = 0
            best_score = np.sqrt(val_score)
        else:
            stop_counter += 1
        if stop_counter >= STOP_NUM:
            break
    best_score_list.append(best_score)